# **Black-Scholes PINN Approximator**
In this notebook, I'll be approximating the **Black-Scholes** equation using a **Physics-Informed Neural Network** or **PINN**. This will be foundational for when we scale up to the **Heston Model**. We will also cross-validate the data in this notebook with the solution found in our RK4 numerical solver and the PINN will be optimized via a **Sweep**.



---

$$
\frac{\partial V}{\partial t} + \frac{1}{2}\sigma^2 S^2 \frac{\partial^2 V}{\partial S^2} + rS \frac{\partial V}{\partial S} - rV = 0
$$
<center>

**Black-Scholes PDE**
</center>

Where:
* $V$: Option price
* $t$: Time
* $S$: Price of the underlying asset
* $\sigma$: Volatility of the underlying asset's returns
* $r$: Risk-free interest rate

---

$$


## **Import Libraries + Set Parameters**

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import ReduceLROnPlateau
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# Manually set random seed for reproducibility
torch.manual_seed(42)
np.random.seed(42)

# Black-Scholes parameters (match with RK4 solver)
r = 0.05          # Risk-free rate
sigma = 0.2       # Volatility
K = 100           # Strike price
T = 1.0           # Time to maturity (in years)
S_max = 250       # Max stock price in spatial domain
S_min = 0         # Min stock price
N = 500           # Number of spatial grid points

# Create spatial grid
S = torch.linspace(S_min, S_max, N).view(-1, 1).requires_grad_()
t = torch.linspace(0, T, N).view(-1, 1).requires_grad_()

## **Model**

In [ ]:
class blackScholesPINN(nn.Module):
    def __init__(self, input_dim=2, output_dim=1, hidden_layers=4, neurons_per_layer=64, activation_fn=nn.Tanh()):
        super().__init__()
        layers = []

        layers.append(nn.Linear(input_dim, neurons_per_layer))
        layers.append(activation_fn)

        for _ in range(hidden_layers - 1):
            layers.append(nn.Linear(neurons_per_layer, neurons_per_layer))
            layers.append(activation_fn)

        layers.append(nn.Linear(neurons_per_layer, output_dim))
        self.net = nn.Sequential(*layers)

    def forward(self, x):
        return self.net(x)


def initialize_weights(model, method='xavier'):
    for m in model.modules():
        if isinstance(m, nn.Linear):
            if method == 'xavier':
                nn.init.xavier_uniform_(m.weight)
            elif method == 'kaiming':
                nn.init.kaiming_uniform_(m.weight)
            elif method == 'normal':
                nn.init.normal_(m.weight, mean=0.0, std=0.1)
            if m.bias is not None:
                nn.init.zeros_(m.bias)

## **Losses**

In [ ]:
def pde_loss(model, S, t):
    S.requires_grad_(True)
    t.requires_grad_(True)
    X = torch.cat((S, t), dim=1)
    V = model(X)

    V_t = torch.autograd.grad(V, t, grad_outputs=torch.ones_like(V), create_graph=True)[0]
    V_S = torch.autograd.grad(V, S, grad_outputs=torch.ones_like(V), create_graph=True)[0]
    V_SS = torch.autograd.grad(V_S, S, grad_outputs=torch.ones_like(V_S), create_graph=True)[0]

    residual = V_t + 0.5 * sigma**2 * S**2 * V_SS + r * S * V_S - r * V
    return torch.mean(residual.pow(2))


def boundary_loss(model, t):
    S0 = torch.zeros_like(t)
    S_high = torch.full_like(t, S_max)

    bc_low = model(torch.cat((S0, t), dim=1))
    bc_high = model(torch.cat((S_high, t), dim=1))
    # This is the boundary for a call option
    # It would be K - S_max for a put option
    expected_high = S_max - K * torch.exp(-r * (T - t))

    return torch.mean(bc_low.pow(2)) + torch.mean((bc_high - expected_high).pow(2))


def initial_loss(model, S):
    t0 = torch.zeros_like(S)
    X0 = torch.cat((S, t0), dim=1)

    V_pred = model(X0)
    V_true = torch.clamp(S - K, min=0.0)

    return torch.mean((V_pred - V_true).pow(2))


## **Training Parameters**

In [ ]:
epochs = 7500

configs = {
    'hidden_layers': 6,
    'neurons_per_layer': 32,
    'activation': 'relu',
    'init_method': 'xavier',
    'initial_lr': 0.005,
    'pde_weight_scale': 14.46,
    'bc_weight_scale': 0.411,
    'ic_weight_scale': 0.086
}

loss_history = []
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=20, min_lr=1e-7, verbose=True)

def train_pinn():
    model = blackScholesPINN(
        input_dim=2,
        output_dim=1,
        hidden_layers=configs['hidden_layers'],
        neurons_per_layer=configs['neurons_per_layer'],
        activation_fn=getattr(nn, configs['activation'])()
    )

    initialize_weights(model, method=configs['init_method'])

    optimizer = optim.Adam(model.parameters(), lr=configs['initial_lr'])
    scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=100, verbose=True)

    S = torch.linspace(0.0, S_max, 100).view(-1, 1)
    t = torch.linspace(0.0, T, 100).view(-1, 1)

    for epoch in range(epochs):
        model.train()
        optimizer.zero_grad()

        loss_pde = pde_loss(model, S, t) * configs['pde_weight_scale']
        loss_bc = boundary_loss(model, t) * configs['bc_weight_scale']
        loss_ic = initial_loss(model, S) * configs['ic_weight_scale']

        loss = loss_pde + loss_bc + loss_ic
        loss.backward()
        optimizer.step()

        if epoch % 500 == 0:
            print(f'Epoch {epoch}, Loss: {loss.item()}')

        scheduler.step(loss)

    return model


## **Plot Loss**

In [ ]:
plt.plot()